In [1]:
import pandas as pd
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy

In [2]:
df_tweets=pd.read_csv('tweets2_nogreenpass_vaccinatevi_casapound.csv')

In [3]:
df_retweeted=pd.read_csv('retweetted2_nogreenpass_vaccinatevi_casapound.csv')

In [4]:
retweeted=df_retweeted.merge(df_tweets[['id_retweet','user_id_retweet','text_complete','hashtags','colours']],left_on=['id','author_id'],right_on=['id_retweet','user_id_retweet'],how='left')[['id', 'author_id', 'author','n_rt','text_complete','hashtags', 'colours_y']].reset_index(drop=True)

In [5]:
retweeted=retweeted.drop_duplicates(['id','author_id','author']).reset_index(drop=True)

In [6]:
retweeted.head()

,id,author_id,author,n_rt,text_complete,hashtags,colours_y
0,1476028900312551426,2963478049,singerindubai,1,Ho appena visto #DontLookUp incredibile che lo...,['#NoGreenPass'],#fb1239
1,1475926285365006345,1364298062919794688,Lorenzo62752880,106,Anche il sindaco di Verona positivo nonostante...,['#NoGreenPass'],#fb1239
2,1476117524446130180,1296390431190679553,1nessuno100mil2,1,"In Italia si respira una tensione enorme, i n...",['#NoGreenPass'],#fb1239
3,1475960153648291841,1238800491632607232,gianluca826,60,Bassano del Grappa non si ferma. Resistenza do...,['#NoGreenPass'],#fb1239
4,1476095815525584896,1453282570582568960,IlDisilluso,19,"""Sono stati riportati più effetti avversi per ...",[],#aaaacc


In [8]:
Nodes_df6=pd.read_csv('Nodes_df6.csv')

In [9]:
Nodes_df6.columns

Index(['Id', 'Label', 'Interval', 'Colour'], dtype='object')

In [11]:
retweetted_joined=retweeted.merge(Nodes_df6[['Id','Label','Colour']],left_on='author_id',right_on='Id',how='left')

In [27]:
lista_correzione=['#nogreenpass','#nogreenpass','#vaccinatevi','#nogreenpass','#nogreenpass','#nogreenpass','#nogreenpass','#nogreenpass','#nogreenpass','#nogreenpass','#nogreenpass','#nogreenpass','#vaccinatevi']

index_correzione=retweetted_joined[(retweetted_joined['Colour'].isna()) & (retweetted_joined['hashtags']=='[]')].index

In [35]:
for iter,index in enumerate(index_correzione):
    retweetted_joined.loc[index,'hashtags']=lista_correzione[iter]

In [38]:
for index, row in retweetted_joined.iterrows():
    if row['Colour'] in ['#fb1239','#99cccc']:
        retweetted_joined.at[index,'hashtags']='#NoGreenPass'
        retweetted_joined.at[index,'colours_y']=0
    if row['Colour'] in ['#007474','#b00b69']:
        retweetted_joined.at[index,'hashtags']='#vaccinatevi'
        retweetted_joined.at[index,'colours_y']=1
    if row['Colour'] in ['#042069']:
        retweetted_joined.at[index,'hashtags']='#casapound'
        retweetted_joined.at[index,'colours_y']=2

In [49]:
retweeted=retweetted_joined

In [39]:
nlp = spacy.load('it_core_news_lg') #, disable=['parser', 'ner'])

In [50]:
retweeted['text_preprocessed']=retweeted['text_complete'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [51]:
#pulizia da simboli
new_text=[]
for index,row in retweeted.iterrows():
    l1=[ t for t in row['text_preprocessed'].split() if t.startswith('#') ]
    l2=[ t for t in row['text_preprocessed'].split() if t.startswith('http') ]
    l3=[ t for t in row['text_preprocessed'].split() if t.startswith('@') ]
    stopword=l1+l2+l3
    querywords=row['text_preprocessed'].split()
    resultwords=[word for word in querywords if word.lower() not in stopword]
    result = ' '.join(resultwords)
    new_text.append(result)

retweeted['text_preprocessed']=new_text

In [52]:
#vengono cancellati tutti i termini che non sono alfanumerici o che non corrispondono a spazi vuoti
retweeted['text_preprocessed'] = retweeted['text_preprocessed'].str.replace('[^\w\s]',' ')


C:\Users\Eric\AppData\Local\Temp/ipykernel_23148/2839750736.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  retweeted['text_preprocessed'] = retweeted['text_preprocessed'].str.replace('[^\w\s]',' ')


In [63]:
##pulizia da emoji e lemmatization usando la library Spacy
#result=[]
#for i in retweeted.index:
#    stopword=['u001f600','u0001f64f','u0001f5ff','u0001f300','u0001f680','u0001f6ff','u0001f1e0','u0001f1ff','u00002702','u000027b0','u000024c2','u0001f251']
#    querywords=retweeted['text_preprocessed'][i].split()
#    resultwords=[word for word in querywords if word.lower() not in stopword]
#    result.append(' '.join(resultwords))
#    
#retweeted['text_preprocessed']=result
#
#stop = stopwords.words('italian')
#
#retweeted['text_preprocessed'] = retweeted['text_preprocessed'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))


In [53]:
def space(comment):
    doc = nlp(comment)
    return " ".join([token.lemma_ for token in doc])

retweeted['text_preprocessed']= retweeted['text_preprocessed'].apply(space)

In [16]:
from nltk.sentiment import vader
import nltk

In [21]:
sentix=pd.read_csv("sentix.csv", header=None,delimiter=';')

In [133]:
#riscaliamo i valori della sentiment come i precedenti valori del lexicon vader
#consideriamo valori estremi di vader lexicon -4 e +4
#consideriamo valori estremi di sentix lexicon -1 e +1

with open("sentix.txt",'w',encoding="utf-8") as txt_file:
    for index,line in sentix.iterrows():
        scaled_score=(((float(line[5]) + 1)/2)*(8))-4
        txt_file.write(str(line[0])+'\t'+str(scaled_score)+'\n')

In [22]:
d= dict([(parola,score) for parola, score in zip(sentix[0],sentix[5])])
sentix_dict=d

In [23]:
Analyzer = vader.SentimentIntensityAnalyzer()

Analyzer.lexicon.update(sentix_dict)

In [46]:
from vaderSentiment import vaderSentiment
from vader_sentiment.vader_sentiment import SentimentIntensityAnalyzer

In [143]:
#arricchimento vader lexicon
with open('lexicon_vader_enriched.txt','w',encoding="utf-8") as newfile:
    with open('vader_lexicon.txt') as vader_file:
        for line in vader_file:
            newfile.write(line)
    with open('sentix.txt',encoding="utf-8") as sentix_file:
        for line in sentix_file:
            newfile.write(line)

In [67]:
#non facciamo pulizia del testo sulle emojii
analyzer = vaderSentiment.SentimentIntensityAnalyzer(lexicon_file='lexicon_vader_enriched.txt',emoji_lexicon='emoji_utf8_lexicon.txt')

#il termine "positivo" è stato modificato associandolo a un sentiment negativo
#il termine "negativo" è associato a un sentiment neutro


In [26]:
analyzer2=SentimentIntensityAnalyzer(lexicon_file='sentix.txt',emoji_lexicon='emoji_utf8_lexicon.txt')

In [54]:
testo=retweeted['text_preprocessed'][16]
#testo='positivo'

print(testo)
print('\n')
print('NLTK: ',Analyzer.polarity_scores(testo))
print('VADER-MULTI: ', analyzer.polarity_scores(testo)) #algoritmo migliore
print('VADER: ', analyzer2.polarity_scores(testo))

2022   il futuro essere soltanto nostro il coord   emilia rom   invitare tutto il cittadino venerdì 31 dicembre   alla ora 15 00   presso piazzale jacchia del giardino margherita di per celebrare il nuovo anno in uno evento comporre da parola   teatro   poesia e musica




NameError: name 'Analyzer' is not defined

In [55]:
print('VADER-MULTI: ', analyzer.polarity_scores(testo)) #algoritmo migliore

VADER-MULTI:  {'neg': 0.101, 'neu': 0.791, 'pos': 0.108, 'compound': -0.1779}


In [63]:
retweeted=retweeted.reset_index(drop=True)

In [68]:
retweeted['sentiment']=0.0

for index,row in retweeted.iterrows():
    retweeted.loc[index,'sentiment']=analyzer.polarity_scores(row['text_preprocessed'])['compound']


HTTPError: HTTP Error 429: Too Many Requests